In [ ]:
import numpy as np
import pandas as pd
train=pd.read_csv("/Users/saksham/Desktop/train.csv")
xTest=pd.read_csv("/Users/saksham/Desktop/test.csv")
yTrain=np.array(train["label"])
xTrain=np.array(train.drop(["label"],axis=1))
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
a=unpickle("/Users/saksham/Desktop/cifar-10-batches-py/test_batch")
y=np.array(a)
yTest=np.array(a[b'labels'])
meanImage = (np.mean(xTrain, axis=0)).astype(int)
xTrain -= meanImage
xTest -= meanImage

# Reshape data from channel to rows
xTrain = np.reshape(xTrain, (xTrain.shape[0], -1))
xTest = np.reshape(xTest, (xTest.shape[0], -1))

# Add bias dimension columns
xTrain = np.hstack([xTrain, np.ones((xTrain.shape[0], 1))])
xTest = np.hstack([xTest, np.ones((xTest.shape[0], 1))])



In [ ]:

class Svm (object):

    def __init__ (self, inputDim, outputDim):
        self.W = None

        sigma =0.01
        self.W = sigma * np.random.randn(inputDim,outputDim)


    def calLoss (self, x, y, reg):

        loss = 0.0
        dW = np.zeros_like(self.W)

        s = x.dot(self.W)
        #Score with yi
        s_yi = s[np.arange(x.shape[0]),y]
        #finding the delta
        delta = s- s_yi[:,np.newaxis]+1
        #loss for samples
        loss_i = np.maximum(0,delta)
        loss_i[np.arange(x.shape[0]),y]=0
        loss = np.sum(loss_i)/x.shape[0]
        #Loss with regularization
        loss += reg*np.sum(self.W*self.W)
        #Calculating ds
        ds = np.zeros_like(delta)
        ds[delta > 0] = 1
        ds[np.arange(x.shape[0]),y] = 0
        ds[np.arange(x.shape[0]),y] = -np.sum(ds, axis=1)

        dW = (1/x.shape[0]) * (x.T).dot(ds)
        dW = dW + (2* reg* self.W)
        return loss, dW

    def train (self, x, y, lr=1e-3, reg=1e-5, iter=100, batchSize=10000):


        lossHistory = []
        for i in range(iter):
            xBatch = None
            yBatch = None

            num_train = np.random.choice(x.shape[0], batchSize)
            xBatch = x[num_train]
            yBatch = y[num_train]
            loss, dW = self.calLoss(xBatch,yBatch,reg)
            self.W= self.W - lr * dW
            lossHistory.append(loss)

            if  i % 1 == 0 and len(lossHistory) != 0:
                print ('Loop {0} loss {1}'.format(i, lossHistory[i]))

        return lossHistory

    def predict (self, x,):

        yPred = np.zeros(x.shape[0])

        s = x.dot(self.W)
        yPred = np.argmax(s, axis=1)

        return yPred


    def calAccuracy (self, x, y):
        acc = 0

        yPred = self.predict(x)
        acc = np.mean(y == yPred)*100

        return acc
numClasses = np.max(yTrain) + 1

print ('Start training Svm classifier')



In [ ]:
classifier = Svm(xTrain.shape[1], numClasses)



In [ ]:
# Training classifier
classifier.train(xTrain, yTrain, lr=5e-7, reg=5e4, iter=100)
print ('Training acc:   {0}%'.format(classifier.calAccuracy(xTrain, yTrain)))
print ('Testing acc:    {0}%'.format(classifier.calAccuracy(xTest, yTest)))